# V4_2の最終的に使うmodel選定のために，どのようにモデルを切り抜いたらいいかを分析するシート
＝現在の方針としては昨年同月でtest,finalともに利益が出ているモデルを切り抜いて，その連続性から安定を判別する．

In [1]:
# import warnings
# warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt

import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
from sklearn.model_selection import train_test_split
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
import time
import datetime
from dateutil.relativedelta import relativedelta
import os #ディレクトリ作成用
import xgboost as xgb
import sys
from sklearn.preprocessing import StandardScaler#モデルの評価用に標準化する関数
import scipy.stats#モデルの評価用に標準化する関数
import warnings

#自作のモジュールのインポート
#sys.path.append("..")
import module.master as master
import module.modeling_scores as modeling


In [3]:
#使えそうな関数
def get_season_date(now_date):#日付(datetime型)を渡すと，その日付で購買予測を行う際に使用するデータの区間を返す関数
    use_data_year=now_date.year
    if (now_date.month>=1)and(now_date.month<4):
        use_data_month=1
    elif (now_date.month>=4)and(now_date.month<7):
        use_data_month=4
    elif (now_date.month>=7)and(now_date.month<10):
        use_data_month=7
    elif (now_date.month>=10):
        use_data_month=10
    else:
        print('what???????')
    use_data_date=datetime.datetime(year=use_data_year, month=use_data_month,day=1)
    return use_data_date


def get_4_section_dt(now_date):#今いる区間から直近4区間の開始日をリストで返してくれる関数
    now_sec_date=get_season_date(now_date)
    diff_sec_stdates=[0]*4
    for i in range(len(diff_sec_stdates)):
        diff_sec_stdates[3-i]=now_sec_date- relativedelta(months=3*(i+1))#古い順に日付を入れていく
    return diff_sec_stdates

def get_feature_weights(score_final_df):#相関係数を変数の重みとして取得する関数，（1comあたり1行のdfを返す．）(引数として一つ前の区間,もしくは昨年同月区間のfinalありスコアのdfを渡す)
    plus_m_th=1
    num_hit_th=3
    feature_weights_df=pd.DataFrame()
    for com in sorted(score_final_df['target_com'].value_counts().sort_index().index):
        target_com_df=score_final_df[score_final_df['target_com']==com]
        #ざっと切り捨て，あまりに少ないのは適さないと思うので消す
        target_com_df=target_com_df[target_com_df['plus_month_num_test']>=plus_m_th].copy()#収益がプラスだった月の数で切り捨てる
        target_com_df=target_com_df[target_com_df['num_hit_test']>=num_hit_th].copy()#的中した予測の数で切り捨てる，大体一カ月に一件くらいをイメージ
        target_com_corr=target_com_df.corr()
        target_com_weights=target_com_corr.loc['gain_final']['total_get_test':'diff_mea_med_test']#相関係数を代用した変数ごとの重みづけ        
        feature_weights_df=feature_weights_df.append(target_com_weights)
    feature_weights_df['target_com']=score_final_df['target_com'].value_counts().sort_index().index
    feature_weights_df=feature_weights_df.set_index('target_com')
    feature_weights_df=feature_weights_df.fillna(0)
    return feature_weights_df


def add_std_score(good_model_df,feature_weights):#偏差値と相関係数をもとにしたスコアを与える
    add_score_df=pd.DataFrame()
    for com in sorted(good_model_df['target_com'].value_counts().sort_index().index):
        target_com_df=good_model_df[good_model_df['target_com']==com]
        #display(feature_weights)
        for f_name,weight in feature_weights.items():
            target_com_df.loc[:,'{}_score'.format(f_name)]=scipy.stats.zscore(target_com_df['{}'.format(f_name)].values)
            target_com_df.loc[:,'{}_score'.format(f_name)]=target_com_df['{}_score'.format(f_name)]*weight
        target_com_df=target_com_df.fillna(0)
        target_com_df['score']=0
        for f_name in feature_weights.index:
            target_com_df['score']=target_com_df['score']+target_com_df['{}_score'.format(f_name)]
        #display(target_com_df)
        add_score_df=pd.concat([add_score_df, target_com_df], axis=0)#きめのパラメータで切り落としたので結合
    
    return add_score_df

def add_std_score_DFversion(good_model_df,feature_weights_df):#偏差値と相関係数をもとにしたスコアを与える
    add_score_df=pd.DataFrame()
    for com in sorted(good_model_df['target_com'].value_counts().sort_index().index):
        target_com_df=good_model_df[good_model_df['target_com']==com]
        feature_weights=feature_weights_df.loc[com].copy()
        #display(feature_weights)
        for f_name,weight in feature_weights.items():
            target_com_df.loc[:,'{}_score'.format(f_name)]=scipy.stats.zscore(target_com_df['{}'.format(f_name)].values)
            target_com_df.loc[:,'{}_score'.format(f_name)]=target_com_df['{}_score'.format(f_name)]*weight
        target_com_df=target_com_df.fillna(0)
        target_com_df['score']=0
        for f_name in feature_weights.index:
            target_com_df['score']=target_com_df['score']+target_com_df['{}_score'.format(f_name)]
        #display(target_com_df)
        add_score_df=pd.concat([add_score_df, target_com_df], axis=0)#きめのパラメータで切り落としたので結合
    
    return add_score_df



def select_use_model(model_std_score_df):#相関係数と標準偏差によって与えられたscoreをもとに最終的に使用するモデルを選定する関数
    use_model_df=pd.DataFrame()
    for com in sorted(model_std_score_df['target_com'].value_counts().index):
        target_com_df=model_std_score_df[model_std_score_df['target_com']==com].copy()
        com_selected_row=target_com_df.sort_values('score', ascending=False).iloc[:1]
        #use_model_df=pd.concat([use_model_df, com_selected_row], axis=0)#きめのパラメータで切り落としたので結合
        use_model_df=use_model_df.append(com_selected_row,ignore_index=True)
    return use_model_df


def twin_plt(x,y1,y2,x_n,y1_n,y2_n):#2本軸でのグラフの描写
    fig = plt.figure(figsize=(19,8))
    plt.tick_params(colors='black')
    plt.rcParams["font.size"] = 18
    
    ax1 = fig.add_subplot()
    fig.set_facecolor(color='white')
    ax1.plot(x, y1,label=y1_n, marker="o")
    ax1.tick_params()
    plt.ylim(80, 200)
    ax2 = ax1.twinx()
    #print(y2)
    ax2.plot(x, y2,label=y2_n,color='red', marker="*")
    ax1.axhline(100,ls = "-.",color='green')
    #ax2.tick_params(colors='white')
    ax2.tick_params()

    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1+h2, l1+l2, loc='lower center')

    #ax1.set_xlabel(x_n,color='white')
    #ax1.set_ylabel(y1_n,color='white')
    #ax2.set_ylabel(y2_n,color='white')
    ax1.set_xlabel(x_n)
    ax1.set_ylabel(y1_n)
    ax2.set_ylabel(y2_n)
    plt.show()
    return None

## 新規関数　

In [9]:
def serch_group(models_df):#モデルシートのパラメータをもとに，パラメータが連続している範囲内で同じ集団とし，集団番号を与える関数
    add_number_df=models_df.copy()
    add_number_df["group_num"]=0
    g_num=0
    g_num_arr=[0]*len(add_number_df)
    diff_index=-1
    i=0
    for index,row in add_number_df.iterrows():
        if index==diff_index+1:
            g_num_arr[i]=g_num
            diff_index=index#現在のインデックスを一つ前として保存
        elif index>diff_index+1:
            g_num=1+g_num
            g_num_arr[i]=g_num
            diff_index=index#現在のインデックスを一つ前として保存
        else:
            print('???????????????????????????????????')
        i+=1
    add_number_df["group_num"]=g_num_arr
    return add_number_df




In [10]:
version='V4_2'#バージョン
#use_modelの書き込み先作成のための箱作り
place_master=master.get_place_master()
for place in place_master.items():
    #print(place[0],place[1],'\n')
    place_name=place[1]
    dir_path = "../bot_database/{place_name}/model_score_{place_name}/use_model/{version}".format(place_name=place_name,version=version)#作成したデータの書き込み先#使用するデータの読み込み
    if os.path.exists(dir_path)==False:
        os.makedirs(dir_path)
    else:
        pass

# モデルの選定のために，集団のサイズがどれくらい影響数rのか，また集団サイズの分布について分析する

In [83]:
place_master=master.get_place_master()
place_names=[place_name for place_name in place_master.values()]#会場名のみを収納した配列
#絶対に固定のパラメータたち
plus_m_th=1

num_hit_th=3
version='V4_2'
mode='model_score_period_'
mode_type='period'
#today=datetime.date.today()#今日の日付の取得
today=datetime.datetime(year=2021, month=10,day=1)
now_date=today
for place_name in place_names:
    #place_name='kiryu'
    print(place_name,'===============================================================')
    diff_sec_dates=get_4_section_dt(now_date)#直近４区間の開始日をリストで取得
    now_date_txt=now_date.strftime('%Y%m%d')
    diff_sec_dates=diff_sec_dates[1:]#テスト時のみの特別処理！！データが１区間だけ足りないので今回はスルーする
    #閾値にあてはまった　モデルたちがどんな奴らなのかを確認する
    for i in range(len(diff_sec_dates)):
        #i=0
        sec_date=diff_sec_dates[i]
        sec_date_txt=sec_date.strftime('%Y%m%d')

        #昨年同月のfinalあり分析結果のシートから使えそうなcom，集団を作成し，パラメータが参照できるように切り抜く
        date_cal_last=sec_date- relativedelta(months=12)#昨年同月の区間の日付を切り抜く
        date_cal_last_txt=date_cal_last.strftime('%Y%m%d')
        dir_path='../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{place_name}_{mode}st{now_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,now_ym=date_cal_last_txt,version=version)
        model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
        model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
        #並べる順番を変える，閾値をはじめの方に持ってきた
        model_score_df=model_score_df.sort_values(['target_com', 'threshold','depth','target_per'])
        model_score_df = model_score_df.reset_index(drop=True)
        #print('before_drop[num_com]:',len(model_score_df))#収益で切り落とす前のモデル数
        both_plus_models=model_score_df[(model_score_df['gain_test']>100)&(model_score_df['gain_final']>100)].copy()#まずは昨年同月の，testとfinalともに利益が出ていたものを切り抜く
        #print('after_drop[num_com]:',len(both_plus_models))#収益で切り落とした後のモデル数
        models_has_g_num_df=serch_group(both_plus_models)#モデルのパラメータから集団番号を付与
        
        
        #試しに決めの値で集団長を指定して切り抜く
        len_th=7
        g_num_len=models_has_g_num_df['group_num'].value_counts()#集団番号ごとの長さ
        numbers_g_s=g_num_len.loc[g_num_len>=len_th]#集団の長さが閾値以上のものをvalue_counts()の結果から切り抜く
        group_df=pd.DataFrame()
        for g_num in numbers_g_s.index:
            target_g_num_df=models_has_g_num_df[models_has_g_num_df['group_num']==g_num]
            group_df=pd.concat([group_df,target_g_num_df])
        g_nums_counts=group_df['group_num'].value_counts(sort=False)
        #集団の真ん中のモデルを切り抜く
        center_models_df=pd.DataFrame(columns=group_df.columns)#真ん中で切り抜いたモデルのか格納先
        for index,len_group in g_nums_counts.items():
            model_groups_df=group_df[group_df['group_num']==index].copy()
            target_row=model_groups_df.iloc[int(len_group)-1]
            center_models_df=center_models_df.append(target_row)

        #絶対的な的中数でさらにモデルを切り抜く
        hit_th=5
        good_models_df=center_models_df[(center_models_df['num_hit_test']>hit_th) & (center_models_df['num_hit_final']>hit_th)].copy()
        #good_models_df=center_models_df[(center_models_df['num_hit_test']>2) & (center_models_df['num_hit_final']>2)].copy()
        selected_models_df=pd.DataFrame(columns=group_df.columns)#真ん中で切り抜いたモデルのか格納先
        for com in good_models_df['target_com'].value_counts().index:
            good_models_com_df=good_models_df[good_models_df['target_com']==com].copy()
            good_models_com_row=good_models_com_df.sort_values(['num_hit_final','gain_final'],ascending=[False, False]).iloc[0]#的中数，finalゲインでsp－として一番大きいものにする
            selected_models_df=selected_models_df.append(good_models_com_row)
            
            
        #今年のスコアシートと，昨年同月から選んだモデルたちを渡して，今年で使うモデルを切り抜く
        dir_path='../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{place_name}_{mode}st{now_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,now_ym=sec_date_txt,version=version)
        now_model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')#今年のスコアシートの読み込み
        now_model_score_df=now_model_score_df.drop(["Unnamed: 0"],axis=1)
        use_models_df=get_same_para_model(selected_models_df,now_model_score_df)
        display(use_models_df)

        #display(use_models_df['gain_final'])
    
    
    

kiryu ===============================================================


,target_com,depth,target_per,threshold,total_get_test,total_use_test,num_com_test,num_pred_test,gain_test,gain_std_test,...,total_use_final,num_com_final,num_pred_final,gain_final,gain_std_final,num_hit_final,buy_hit_per_final,buy_hit_per_std_final,plus_month_num_final,diff_mea_med_final
6819,14.0,7.0,86.0,0.5,22030.0,25600.0,9.0,256.0,86.054688,57.880980,...,26700.0,11.0,267.0,82.097378,66.736176,9.0,3.370787,0.990918,1.0,192.222222
11139,23.0,5.0,98.0,0.5,26700.0,23000.0,7.0,230.0,116.086957,129.190706,...,25800.0,5.0,258.0,64.341085,42.116694,5.0,1.937984,0.223779,1.0,0.000000


,target_com,depth,target_per,threshold,total_get_test,total_use_test,num_com_test,num_pred_test,gain_test,gain_std_test,...,total_use_final,num_com_final,num_pred_final,gain_final,gain_std_final,num_hit_final,buy_hit_per_final,buy_hit_per_std_final,plus_month_num_final,diff_mea_med_final
2095,5.0,5.0,107.0,0.54,13850.0,17100.0,25.0,171.0,80.994152,29.772926,...,15400.0,30.0,154.0,91.558442,13.826477,18.0,11.688312,1.298701,1.0,98.055556
6813,14.0,7.0,84.0,0.50,21150.0,24700.0,15.0,247.0,85.627530,65.176324,...,21500.0,13.0,215.0,142.697674,144.440827,7.0,3.255814,0.710477,2.0,-45.833333
4323,9.0,7.0,94.0,0.50,22340.0,22600.0,20.0,226.0,98.849558,69.488000,...,20900.0,11.0,209.0,66.650718,43.716351,9.0,4.306220,1.265910,1.0,130.555556


,target_com,depth,target_per,threshold,total_get_test,total_use_test,num_com_test,num_pred_test,gain_test,gain_std_test,...,total_use_final,num_com_final,num_pred_final,gain_final,gain_std_final,num_hit_final,buy_hit_per_final,buy_hit_per_std_final,plus_month_num_final,diff_mea_med_final
1224,3.0,5.0,153.0,0.5,9350.0,8700.0,21.0,87.0,107.471264,76.233764,...,8000.0,18.0,80.0,24.125,32.773118,2.0,2.5,1.443376,0.0,0.0


toda ===============================================================


...

,target_com,depth,target_per,threshold,total_get_test,total_use_test,num_com_test,num_pred_test,gain_test,gain_std_test,...,total_use_final,num_com_final,num_pred_final,gain_final,gain_std_final,num_hit_final,buy_hit_per_final,buy_hit_per_std_final,plus_month_num_final,diff_mea_med_final
2541,6.0,5.0,88.0,0.5,28080.0,36500.0,30.0,365.0,76.931507,70.368507,...,27700.0,13.0,277.0,41.696751,21.214258,11.0,3.971119,0.551453,0.0,43.166667


omura ===============================================================


,target_com,depth,target_per,threshold,total_get_test,total_use_test,num_com_test,num_pred_test,gain_test,gain_std_test,...,total_use_final,num_com_final,num_pred_final,gain_final,gain_std_final,num_hit_final,buy_hit_per_final,buy_hit_per_std_final,plus_month_num_final,diff_mea_med_final
8430,17.0,7.0,119.0,0.5,19580.0,16000.0,13.0,160.0,122.375000,123.886957,...,16800.0,4.0,168.0,12.380952,11.103761,2.0,1.190476,0.343661,0.0,0.000000
1281,3.0,7.0,88.0,0.5,29040.0,28600.0,29.0,286.0,101.538462,42.768948,...,28500.0,22.0,285.0,73.859649,14.834102,17.0,5.964912,0.405158,0.0,287.238095
8901,18.0,7.0,108.0,0.5,23170.0,17400.0,11.0,174.0,133.160920,51.186413,...,20400.0,7.0,204.0,63.333333,38.475183,6.0,2.941176,0.490196,1.0,293.333333


,target_com,depth,target_per,threshold,total_get_test,total_use_test,num_com_test,num_pred_test,gain_test,gain_std_test,...,total_use_final,num_com_final,num_pred_final,gain_final,gain_std_final,num_hit_final,buy_hit_per_final,buy_hit_per_std_final,plus_month_num_final,diff_mea_med_final
2343,5.0,7.0,106.0,0.5,12000.0,12900.0,43.0,129.0,93.023256,25.457649,...,12600.0,28.0,126.0,52.936508,45.978344,8.0,6.349206,2.424643,0.0,-8.333333


,target_com,depth,target_per,threshold,total_get_test,total_use_test,num_com_test,num_pred_test,gain_test,gain_std_test,...,total_use_final,num_com_final,num_pred_final,gain_final,gain_std_final,num_hit_final,buy_hit_per_final,buy_hit_per_std_final,plus_month_num_final,diff_mea_med_final
2920,6.0,7.0,130.0,0.54,5400.0,4700.0,34.0,47.0,114.893617,78.692811,...,6000.0,23.0,60.0,57.0,62.298254,5.0,8.333333,0.96225,1.0,0.0


In [82]:
place_name

'omura'

In [61]:
def serch_group(models_df):#モデルシートのパラメータをもとに，パラメータが連続している範囲内で同じ集団とし，集団番号を与える関数
    add_number_df=models_df.copy()
    add_number_df["group_num"]=0
    g_num=0
    g_num_arr=[0]*len(add_number_df)
    diff_index=-1
    i=0
    for index,row in add_number_df.iterrows():
        if index==diff_index+1:
            g_num_arr[i]=g_num
            diff_index=index#現在のインデックスを一つ前として保存
        elif index>diff_index+1:
            g_num=1+g_num
            g_num_arr[i]=g_num
            diff_index=index#現在のインデックスを一つ前として保存
        else:
            print('???????????????????????????????????')
        i+=1
    add_number_df["group_num"]=g_num_arr
    return add_number_df


def get_same_para_model(selected_model_df,now_year_score_df):
    use_models_df=pd.DataFrame()
    for index,row in selected_model_df.iterrows():
        target_df=now_year_score_df[now_year_score_df["target_com"]==row['target_com']].copy()
        target_df=target_df[target_df['threshold']==row['threshold']].copy()
        target_df=target_df[target_df['depth']==row['depth']].copy()
        target_df=target_df[target_df['target_per']==row['target_per']].copy()
        use_models_df=pd.concat([use_models_df,target_df])
    return use_models_df

In [ ]:
serch_group(both_plus_models)

In [ ]:
group_df[50:100]